### Imporitng Libraries:

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, Flatten
from kerastuner import RandomSearch

In [14]:
data = keras.datasets.fashion_mnist

In [15]:
(X_train, y_train), (X_test, y_test) = data.load_data()

In [16]:
X_train.shape

(60000, 28, 28)

In [17]:
X_train = X_train/250.0
X_test = X_test/250.0

In [18]:
# reshape the train & test dataset
X_train = X_train.reshape(len(X_train), 28, 28, 1)
X_test = X_test.reshape(len(X_test), 28, 28, 1)

In [19]:
# create a function for Model building
def create_model(hp):
  model = Sequential(
      # Creating 1st Conv2D layer
      [Conv2D(filters=hp.Int("conv_1_filter", min_value=16, max_value=128, step=16),
              kernel_size=hp.Choice("conv_1_kernel", values=[3,5]),
              activation='relu',
              kernel_initializer='he_normal',
              input_shape=(28, 28, 1)),
      
      # Creating 2nd Conv2D layer
      Conv2D(filters=hp.Int("conv_2_filter", min_value=16, max_value=128, step=16),
             kernel_initializer='he_normal',
             kernel_size=hp.Choice("conv_2_kernel", values=[3,5]),
             activation='relu',
             input_shape=(28, 28, 1)),

      # adding Flatten layer 
      Flatten(),

      # Creating 1st hidden layer 
      Dense(units=hp.Int("layer_1_units", min_value=32, max_value=512, step=32),
            activation='relu',
            kernel_initializer='he_normal'),
      
      # Creating output layer
       Dense(units=10, activation='softmax')
       ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model


In [20]:
tuner = RandomSearch(create_model, objective='val_accuracy', max_trials=5, directory='log', project_name='CNN')
tuner.search(X_train, y_train, epochs=5, validation_split=0.1)

INFO:tensorflow:Reloading Oracle from existing project log/CNN/oracle.json
INFO:tensorflow:Reloading Tuner from log/CNN/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [21]:
model = tuner.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        25632     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               3965184   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 3,993,706
Trainable params: 3,993,706
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(X_train, y_train, epochs=10, validation_split=0.3, validation_data=(X_test, y_test))

Epoch 1/10
1313/1313 [==============================] - 13s 9ms/step - loss: 0.0925 - accuracy: 0.9658 - val_loss: 0.1821 - val_accuracy: 0.9397
Epoch 2/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0559 - accuracy: 0.9797 - val_loss: 0.1944 - val_accuracy: 0.9386
Epoch 3/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0337 - accuracy: 0.9880 - val_loss: 0.2546 - val_accuracy: 0.9351
Epoch 4/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0288 - accuracy: 0.9906 - val_loss: 0.2581 - val_accuracy: 0.9371
Epoch 5/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0204 - accuracy: 0.9928 - val_loss: 0.3090 - val_accuracy: 0.9368
Epoch 6/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0205 - accuracy: 0.9939 - val_loss: 0.3331 - val_accuracy: 0.9304
Epoch 7/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.0178 - accuracy: 0.9940 - val_loss: 0.3978 - val_ac